# R054 - Archisha Sinha
# MBA Tech AI

## Domain: Reinforcement Learning
## Topic: Monte Carlo Prediction - First Visit and Every Visit Algorithm



In [1]:
import numpy as np
import random

In [2]:
# Parameters
rows, cols = 4, 5
states = rows * cols
actions = ['left', 'right', 'up', 'down']
random.seed(42)

In [3]:
# Initialize reward matrix (s1 has +50 reward, others random -5 to +5)
reward_matrix = np.random.randint(-5, 6, size=(rows, cols))
reward_matrix[0][0] = 50  # s1 = 50

In [4]:
# Random policy: for each state, pick a random action
policy_matrix = np.random.choice(actions, size=(rows, cols))

In [5]:
# Environment display
def print_matrix(matrix, title):
    print(f"\n{title}:")
    for row in matrix:
        print(row)

In [6]:
# Action effects (left, right, up, down) as deltas
action_effects = {
    'left': (0, -1),
    'right': (0, 1),
    'up': (-1, 0),
    'down': (1, 0)
}

In [7]:
# Generate random episodes (3 episodes)
def generate_episode():
    episode = []
    current_state = (random.randint(0, rows-1), random.randint(0, cols-1))  # Random start
    while len(episode) < 10:  # Arbitrarily keeping episode length to 10 steps max
        action = policy_matrix[current_state[0]][current_state[1]]
        reward = reward_matrix[current_state[0]][current_state[1]]
        episode.append((current_state, action, reward))

        delta = action_effects[action]
        next_state = (current_state[0] + delta[0], current_state[1] + delta[1])

        # Ensure we don't go out of bounds
        if 0 <= next_state[0] < rows and 0 <= next_state[1] < cols:
            current_state = next_state
        else:
            break
    return episode

In [8]:
# Generate 3 episodes common for both methods
episodes = [generate_episode() for _ in range(3)]

In [9]:
episodes

[[((0, 0), 'left', 50)],
 [((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2)],
 [((1, 1), 'down', -3),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3),
  ((3, 1), 'up', -2),
  ((2, 1), 'down', 3)]]

In [10]:
# Initialize value function for each state
value_first_visit = np.zeros((rows, cols))
value_every_visit = np.zeros((rows, cols))

### First Visit Algorithm

In [11]:
# First-Visit Monte Carlo Prediction
returns_first_visit = {s: [] for s in [(i, j) for i in range(rows) for j in range(cols)]}

for episode in episodes:
    visited = set()
    G = 0
    for (state, action, reward) in reversed(episode):
        G += reward
        if state not in visited:
            returns_first_visit[state].append(G)
            value_first_visit[state] = np.mean(returns_first_visit[state])
            visited.add(state)

### Every Visit Algorithm

In [12]:
# Every-Visit Monte Carlo Prediction
returns_every_visit = {s: [] for s in [(i, j) for i in range(rows) for j in range(cols)]}

for episode in episodes:
    G = 0
    for (state, action, reward) in reversed(episode):
        G += reward
        returns_every_visit[state].append(G)
        value_every_visit[state] = np.mean(returns_every_visit[state])

In [13]:
# Output results
print_matrix(reward_matrix, "Reward Matrix")
print_matrix(policy_matrix, "Policy Matrix")


Reward Matrix:
[50  4 -4  0 -4]
[-3 -3  1 -5  4]
[3 3 4 5 4]
[-2 -2  2  5  4]

Policy Matrix:
['left' 'left' 'down' 'left' 'left']
['right' 'down' 'left' 'right' 'down']
['right' 'down' 'down' 'right' 'down']
['down' 'up' 'down' 'left' 'left']


In [14]:
print("\nGenerated Episodes:")
for i, episode in enumerate(episodes):
    print(f"Episode {i+1}: {episode}")


Generated Episodes:
Episode 1: [((0, 0), 'left', 50)]
Episode 2: [((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2)]
Episode 3: [((1, 1), 'down', -3), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3), ((3, 1), 'up', -2), ((2, 1), 'down', 3)]


In [15]:
print_matrix(value_first_visit, "Value Function (First-Visit MC)")


Value Function (First-Visit MC):
[50.  0.  0.  0.  0.]
[0. 4. 0. 0. 0.]
[0. 2. 0. 0. 0.]
[ 0.  -0.5  0.   0.   0. ]


In [16]:
print_matrix(value_every_visit, "Value Function (Every-Visit MC)")


Value Function (Every-Visit MC):
[50.  0.  0.  0.  0.]
[0. 4. 0. 0. 0.]
[0. 4. 0. 0. 0.]
[0.         1.11111111 0.         0.         0.        ]


## Conclusion

**First-Visit MC Conclusion:**

First-Visit MC converges faster due to processing each state only once per episode, resulting in slightly fewer updates. The values obtained are similar to Every-Visit, but computation is more efficient.

**Every-Visit MC Conclusion:**

Every-Visit MC has higher time complexity as it updates the value function every time a state is visited, leading to more granular value updates. The values are nearly identical, but the method is slower due to repeated updates.